<a href="https://colab.research.google.com/github/Lee-Gunju/AI-paper-code-review-for-personal-project/blob/master/Simple_custom_layer_example_Antirectifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
class Antirectifier(layers.Layer):
    def __init__(self, initializer="he_normal", **kwargs):
        super(Antirectifier, self).__init__(**kwargs)
        self.initializer = keras.initializers.get(initializer)

    def build(self, input_shape):
        output_dim = input_shape[-1]
        self.kernel = self.add_weight(
            shape=(output_dim * 2, output_dim),
            initializer=self.initializer,
            name="kernel",
            trainable=True,
        )

    def call(self, inputs):
        inputs -= tf.reduce_mean(inputs, axis=-1, keepdims=True)
        pos = tf.nn.relu(inputs)
        neg = tf.nn.relu(-inputs)
        concatenated = tf.concat([pos, neg], axis=-1)
        mixed = tf.matmul(concatenated, self.kernel)
        return mixed

    def get_config(self):
        # Implement get_config to enable serialization. This is optional.
        base_config = super(Antirectifier, self).get_config()
        config = {"initializer": keras.initializers.serialize(self.initializer)}
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
# Training parameters
batch_size = 128
num_classes = 10
epochs = 20

# The data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

11493376/11490434 [==============================] - 0s 0us/step
60000 train samples
10000 test samples


In [ ]:
tf.reduce_mean(x_train[0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.13768007>

In [ ]:
tf.reduce_mean(x_train[0], axis=-1, keepdims=True)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.13768007], dtype=float32)>

In [ ]:
tf.reduce_sum(x_train[0])

<tf.Tensor: shape=(), dtype=float32, numpy=107.94118>

In [ ]:
# Build the model
model = keras.Sequential(
    [
        keras.Input(shape=(784,)),
        layers.Dense(256),
        Antirectifier(),
        layers.Dense(256),
        Antirectifier(),
        layers.Dropout(0.5),
        layers.Dense(10),
    ]
)

# Compile the model
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train the model
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.15)

# Test the model
model.evaluate(x_test, y_test)

Epoch 1/20
399/399 [==============================] - 5s 4ms/step - loss: 0.3811 - sparse_categorical_accuracy: 0.8893 - val_loss: 0.2691 - val_sparse_categorical_accuracy: 0.9248
Epoch 2/20
399/399 [==============================] - 2s 4ms/step - loss: 0.1737 - sparse_categorical_accuracy: 0.9524 - val_loss: 0.2646 - val_sparse_categorical_accuracy: 0.9356
Epoch 3/20
399/399 [==============================] - 2s 4ms/step - loss: 0.1357 - sparse_categorical_accuracy: 0.9624 - val_loss: 0.1593 - val_sparse_categorical_accuracy: 0.9630
Epoch 4/20
399/399 [==============================] - 2s 4ms/step - loss: 0.1123 - sparse_categorical_accuracy: 0.9698 - val_loss: 0.1416 - val_sparse_categorical_accuracy: 0.9684
Epoch 5/20
399/399 [==============================] - 2s 4ms/step - loss: 0.0976 - sparse_categorical_accuracy: 0.9751 - val_loss: 0.1542 - val_sparse_categorical_accuracy: 0.9703
Epoch 6/20
399/399 [==============================] - 2s 4ms/step - loss: 0.0920 - sparse_categorica

[0.22567661106586456, 0.9757999777793884]